# 대화 상태 (Conversation State)
OpenAI는 대화 상태를 관리하는 몇 가지 방법을 제공하며, 이는 대화에서 여러 메시지 또는 차례에 걸쳐 정보를 보존하는 데 중요합니다.

### 수동으로 대화 상태 관리하기 

각 텍스트 생성 요청은 독립적이고 상태가 없는(stateless) 요청이지만, 여러 턴에 걸친 대화를 구현하기 위해서는 이전 메시지들을 함께 전달할 수 있습니다.  

이전 메시지 기록을 매번 요청에 포함시켜 전달하면,
모델은 이전 문맥을 기억하며 대화를 이어갈 수 있습니다.

### 대화 상태용 OpenAI API
response API를 사용하면 대화 상태를 자동으로 관리할 수 있습니다.  
Responses API는 상태를 저장하므로 대화 전반의 컨텍스트 관리가 간단한 매개변수로 가능합니다.

In [ ]:
# OpenAI Responses API 예제: previous_response_id 로 대화 이어가기
# 모델이 생성한 첫 번째 응답 출력
# 두 번째 요청: 이전 응답(previous_response_id)을 이어받아 대화 계속
#    → 모델이 직전의 대화 맥락(response.id)을 기억하도록 전달
# 모델이 이전 대화를 기반으로 한 두 번째 응답 출력

### 문서/텍스트를 기반으로 챗봇이 답변하도록 하기

문서/텍스트 내용을 API 호출시 넘겨주면 우리가 원하는 답변을 유도할 수 있습니다.

In [ ]:
# OpenAI Responses API 예제: 콜센터 도우미 시뮬레이션
    # 모델의 역할을 정의하는 system-level 지침
    # 대화의 시작점 (사용자 입력)
# 모델의 첫 번째 응답(초기 인사 또는 안내문) 출력
# 메인 대화 루프: 사용자가 'quit' 입력 시까지 계속 대화 유지
    # 사용자가 'quit'를 입력하면 루프 종료
    # 모델에게 이전 대화 맥락(previous_response_id)을 전달하여 대화 지속
    # 모델의 응답 출력

------------------
(대화 예시)
```
안녕하세요 고객님. 무엇을 도와드릴까요?
사용자 입력:  맞춤법 검사 기능을 사용할 수 있나요?
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 검사 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 해당 필드 내에서 사전 클릭 옵션을 선택하시면 됩니다.
사용자 입력:  인공지능 연구소는 어떤 비밀번호 형식을 요구하나요 
assistant: 인공지능 연구소는 강력한 비밀번호 사용을 권장하며, 비밀번호는 최소 8자 이상이어야 하고 문자와 숫자를 조합해야 합니다.
사용자 입력:  각 섹션의 모든 정보를 제공해야하나요
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 별표(*)로 표시된 모든 항목이 필수 정보로 지정되어 있으므로, 각 섹션의 모든 필수 항목을 반드시 작성해야 합니다.
사용자 입력:  강남에서 남산에 가는 교통편을 알려주세요 
assistant: 강남에서 남산으로 가는 교통편은 여러 가지가 있습니다. 대표적인 방법은 다음과 같습니다:

1. **지하철 이용**:
   - 강남역에서 2호선을 타고 '역삼역' 또는 '선릉역'에서 3호선으로 환승 → 충무로역 또는 명동역에서 하차 → 남산 방향으로 도보 혹은 버스 이용
   - 또는 강남역에서 9호선으로 환승해 '명동역' 또는 '남산촌차고지'로 이동하는 방법도 있습니다.

2. **버스 이용**:
   - 강남 일대에서 남산 순환버스(남산순환버스 또는 시내버스)를 이용할 수 있으며, 일부 버스는 남산 근처 정류장에 정차합니다.

3. **택시 또는 차량 이용**:
   - 택시를 타거나 개인 차량으로 이동하면 약 20~30분 정도 소요됩니다. 남산공원 및 케이블카 하차장 근처에 주차하거나, 케이블카를 이용하려면 남산케이블카 승강장 부근으로 이동하시면 됩니다.

시간과 교통 상황에 따라 최적의 경로를 선택하실 수 있으니, 출발 전에 교통정보 앱이나 지도 서비스를 확인하시는 것을 추천드립니다.
사용자 입력:  quit
```

## 대화 상태를 위한 OpenAI API

OpenAI의 API를 사용하면 각 대화(turn)마다 입력을 수동으로 전달하지 않아도, 대화 상태(conversation state) 를 자동으로 관리할 수 있습니다.

### Conversations API 

Conversations API 는 Responses API 와 함께 작동하여,
지속 가능한 ID를 가진 장기 실행 객체(long-running object) 로서 대화 상태를 유지합니다.
한 번 생성된 대화 객체는 세션, 기기, 작업(job) 간에 계속 사용할 수 있습니다.

Conversations는 메시지, 도구 호출(tool calls), 도구 출력(tool outputs), 기타 데이터 등의 항목(items) 을 저장합니다. 

여러 턴의 대화에서, 이전 응답을 일일이 연결할 필요 없이
이 conversation 객체를 이후 요청에 전달하면 대화 상태와 컨텍스트를 지속적으로 공유할 수 있습니다.